In [1]:
#!pip install seaborn
#!pip install openpyxl
#!pip install sklearn

In [2]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [3]:
file_data = 'SN_m_tot_V2.0.csv'
path_name='../datasets/'
path_name_results='../results/'
file_result = 'Result_SVR_sunspot.csv'

In [4]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')
dataset.columns = ['year','month', 'date','total_sunspot_number','std_derivation','num_observations','def_prov_indicator']

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3295 entries, 0 to 3294
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  3295 non-null   int64  
 1   month                 3295 non-null   int64  
 2   date                  3295 non-null   float64
 3   total_sunspot_number  3295 non-null   float64
 4   std_derivation        3295 non-null   float64
 5   num_observations      3295 non-null   int64  
 6   def_prov_indicator    3295 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 180.3 KB


In [6]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

year                    0
month                   0
date                    0
total_sunspot_number    0
std_derivation          0
num_observations        0
def_prov_indicator      0
dtype: int64

In [7]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
def previsao_MLP(nm_dataset, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['num_observations']=dataset['num_observations']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['num_observations'].shift(n_step)  
      
  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  80% treinam  20% test
  nlinhas = int(np.round(df.shape[0] *0.80)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 1 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 1 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   

  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   
  
  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  sMAPE = round(
		np.mean(
			np.abs(predict - y_test) /
			((np.abs(predict) + np.abs(y_test)))
		)*100, 2
	)

  salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)



In [9]:
#create file to results
criar_arquivo_resultado()

print('forecast for new sunspot number')
for n_time_steps in range(1,13): #predict with 1 to 12 past values of medition
    grid = previsao_MLP('sunspot', dataset, n_time_steps)

forecast for new sunspot number
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-05, 'kernel': 'rbf'}", 1, 137160.14414665214, 370.35137929627336, 165.40125638680337, 44.87842921208547, 31.21, 18.965964794158936]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-05, 'kernel': 'rbf'}", 2, 79421.63602260328, 281.8184451426189, 93.53888803462792, 47.11080569894908, 35.65, 10.21881914138794]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


...

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-06, 'kernel': 'rbf'}", 10, 132329.47347087006, 363.77118284832574, 171.58613875368457, 33.53744311452316, 22.55, 8.275168895721436]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'C': 12550, 'epsilon': 0.01, 'gamma': 1e-05, 'kernel': 'rbf'}", 11, 57529.93045626479, 239.853977361779, 69.04928979916917, 22.44438603541619, 11.29, 7.721556186676025]
Fitting 5 folds for each of 48 candidates, totalling 240 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-05, 'kernel': 'rbf'}", 12, 61440.873311642164, 247.87269577676796, 70.81881708645085, 28.064709963783763, 14.08, 6.693147420883179]


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
